# Training the model
**Table of Contents**
1. [Setup](#1-setup)
2. [Dataset Preparation](#2-dataset-preparation)
3. [Training Loop](#3-training-loop)


## 1. Setup

In [ ]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [ ]:
####################
# Required Modules #
####################

# Generic/Built-in
import random
import sys 
import os

# Libs
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torch.utils.data import DataLoader

In [ ]:
# Add the project root directory to the system path to enable imports from the '/src' folder.

# Get the project directory 
current_dir = os.path.abspath('') # Current '\notebooks' directory
project_dir = os.path.abspath(os.path.join(current_dir, '..')) # Move up one level to project root directory

# Add the project directory to sys.path
sys.path.append(project_dir)

# Move up to project directory
os.chdir(project_dir)
os.getcwd()

# Import custom modules
from src.data_preparation import *
from src.models import *
from src.train_eval import *
from src.utils import *

In [ ]:
# Seeding
SEED = 42

# To be safe, seed all modules for full reproducibility
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # If using CUDA
np.random.seed(SEED)
random.seed(SEED)

## 2. Dataset Preparation

In [ ]:
# Dataset Hyperparameters
sequence_size = 250 
stride = 125
num_train = 32
num_val = 4
num_test = 4

In [ ]:
train_dataset, val_dataset, test_dataset = prepare_datasets(
    sequence_size=sequence_size, 
    stride=stride,
    num_train=num_train,
    num_val=num_val,
    num_test=num_test,
    random_state=SEED, # For reproducibility
    load_if_exists=True
)

In [ ]:
print(f"Train Set: {len(train_dataset)} samples")
print(f"Validation Set: {len(val_dataset)} samples")
print(f"Test Set: {len(test_dataset)} samples")
print(f"Total: {len(train_dataset) + len(val_dataset) + len(train_dataset)} samples")

## 3. Training Loop

In [ ]:
# Hyperparameters
batch_size = 256
learning_rate = 0.001
num_epochs = 40
weight_decay = 1e-5 # L2 Regularization coefficient

In [ ]:
# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

Load the model in the cell below. There are 4 model architectures to choose from: `HarLSTM`, `HarGRU`, `HarTransformer`, and `HarTransformerExperimental`. The default parameter values should suffice.

In [ ]:
# Load model
model_kwargs = {}
model = HarTransformerExperimental(**model_kwargs)

In [ ]:
# Training optimizer
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr = learning_rate,
    weight_decay=weight_decay
)

In [ ]:
# Optional: Continue training by loading trained parameters
trained_params_path = "models/HarTransformerExperimental_2025-04-15_16-28-09/HarTransformerExperimental_best_F1.pth" # Specify path to .pth file here
# model.load_state_dict(torch.load(trained_params_path))

In [ ]:
training_loss_history, validation_loss_history, micro_accuracy_history, macro_accuracy_history, f1_history, precision_history, recall_history, normalizer = train_HAR70_model(
    model, 
    optimizer, 
    train_dataloader, 
    validation_dataloader, 
    num_epochs = num_epochs
)

In [ ]:
save_dir = save_training_plots_and_metric_history(
    training_loss_history, validation_loss_history, micro_accuracy_history, macro_accuracy_history,
    f1_history, precision_history, recall_history, type(model).__name__
)

## 4. Test

In [ ]:
# Test metrics
loss, micro_accuracy, macro_accuracy, f1, precision, recall, conf_matrix = evaluate_HAR70_model(model, test_dataloader, normalizer)
print(f"(Test) Loss: {loss:.4f}, Accuracy (micro): {micro_accuracy:.4f}, Accuracy (macro): {macro_accuracy:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

In [ ]:
compute_metrics_from_confusion_matrix(conf_matrix)

In [ ]:
# Original label mapping (12)
class_names = [
    "Walking", # Label 0
    "Running",
    "Shuffling",
    "Stairs (ascending)",
    "Stairs (descending)",
    "Standing",
    "Sitting",
    "Lying",
    "Cycling (sit)",
    "Cycling (stand)",
    "Cycling (sit, inactive)",
    "Cycling (stand, inactive)" # Label 11
]

# Plot confusion matrix
plot_name = "unnormalized_conf_matrix"
plot_and_save_confusion_matrix(
    save_dir=save_dir,
    conf_matrix=conf_matrix,
    file_name=plot_name,
    class_names=class_names
)
metric_results = save_and_compute_metrics_from_confusion_matrix(
    save_dir=save_dir,
    conf_matrix=conf_matrix,
    file_name=plot_name
)
metric_results

In [ ]:
# Normalize confusion matrix (row-wise)
conf_matrix_normalized = normalize_confusion_matrix(conf_matrix)

# Plot confusion matrix
plot_name = "normalized_conf_matrix"
plot_and_save_confusion_matrix(
    save_dir=save_dir,
    conf_matrix=conf_matrix_normalized,
    file_name=plot_name,
    class_names=class_names
)
metric_results = save_and_compute_metrics_from_confusion_matrix(
    save_dir=save_dir,
    conf_matrix=conf_matrix_normalized,
    file_name=plot_name
)
metric_results

Merge and ignore classes for a fairer comparison with HAR70+ paper results.

In [ ]:
print(class_names)
## WIP WIP

In [ ]:
updated_conf_matrix, updated_class_names = ignore_classes(conf_matrix, class_names, [1,8,9,10,11])
updated_class_names

In [ ]:
merge_groups = [
    [0, 2, 3], # Walking + Stairs (ascending) + Stairs (descending)
    [1, 4], # Shuffling + Standing
]
updated_conf_matrix, updated_class_names = merge_multiple_classes(
    conf_matrix=updated_conf_matrix,
    merge_groups=merge_groups,
    merge_names=["Walking (merged)", "Standing (merged)"],
    class_names=updated_class_names
)
updated_class_names

In [ ]:
# Normalize confusion matrix (row-wise)
updated_conf_matrix_normalized = normalize_confusion_matrix(updated_conf_matrix)

# Plot confusion matrix
plot_and_save_confusion_matrix(
    save_dir=save_dir,
    conf_matrix=updated_conf_matrix_normalized,
    file_name="merged_conf_matrix_normalized",
    class_names=updated_class_names
)

In [ ]:
compute_metrics_from_confusion_matrix(updated_conf_matrix)

In [ ]:
# Save session information
save_model_information(
    # Dataset Config:
    save_dir=save_dir, 
    sequence_size=sequence_size, 
    stride=stride, 
    num_train=num_train, 
    num_val=num_val, 
    num_test=num_test, 
    random_state=SEED,
    # Training Hyperparams:
    optimizer_name=type(optimizer).__name__,
    batch_size=batch_size,
    learning_rate=learning_rate,
    num_epochs=num_epochs,
    weight_decay=weight_decay,
    # Model Hyperparams:
    model_kwargs=model_kwargs,
    # Test Results:
    loss=loss,
    micro_accuracy=micro_accuracy,
    macro_accuracy=macro_accuracy,
    f1=f1,
    precision=precision,
    recall=recall
)